In [282]:
!pip install botorch


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [283]:
import time
import torch
import numpy as np
import botorch
from botorch import fit_gpytorch_model
from botorch.models import HeteroskedasticSingleTaskGP
from gpytorch.mlls.sum_marginal_log_likelihood import ExactMarginalLogLikelihood

torch.set_default_dtype(torch.float64)

In [284]:
lower_bounds = [-2, -2]
upper_bounds = [2, 2]
bounds = [lower_bounds, upper_bounds]
bounds = torch.tensor(bounds, dtype=torch.float)


def blackbox_func(x):
    res = x[0]**3 + x[1]**3
    var = torch.rand(1).item()/2 # var in [0.5, 1.5]
    res_noise = np.random.normal(loc=res, scale=np.sqrt(var))
    
    return res_noise, var

blackbox_func([2.0, 2.0])

(16.79338599758987, 0.43713642736128705)

In [285]:
def generate_initial_data(n=10):
    # generate training data
    train_x = torch.rand(n, 2)
    train_y = []
    train_y_var = []

    for i in range(n):
        t_y, t_y_var = blackbox_func(train_x[i])
        train_y.append(t_y)
        train_y_var.append(t_y_var)

    train_y = torch.tensor(train_y).reshape(-1,1)
    train_y_var = torch.tensor(train_y_var).reshape(-1,1)


    return train_x, train_y, train_y_var
    
    
def initialize_model(train_x, train_y, train_y_var, state_dict=None):
    # define models for objective and constraint
    model = HeteroskedasticSingleTaskGP(train_x, train_y, train_y_var)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)

    # load state dict if it is passed
    if state_dict is not None:
        model.load_state_dict(state_dict)
        
    return mll, model

In [286]:
from botorch.optim import optimize_acqf

# def optimize_acqf_and_get_observation(acq_func):
#     """Optimizes the acquisition function, and returns a new candidate and a noisy observation."""
#     # optimize
#     candidates = joint_optimize(
#         acq_function=acq_func,
#         bounds=bounds,
#         q=1,
#         # num_restarts=10,
#         # raw_samples=500,  # used for intialization heuristic
#     )

#     # observe new values 
#     new_x = candidates.detach()
#     new_y, new_y_var = blackbox_func(new_x)
#     return new_x, new_y, new_y_var


In [287]:
from botorch import fit_gpytorch_model
from botorch.acquisition.monte_carlo import qNoisyExpectedImprovement
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling import IIDNormalSampler, SobolQMCNormalSampler

def get_next_points(train_x, train_y, train_y_var, best_y, bounds, n_points):    
    train_x_mean = torch.mean(train_x, dim=0)
    train_x_std = torch.std(train_x, dim=0)
    train_x = (train_x-train_x_mean)/train_x_std


    train_y_mean = torch.mean(train_y, dim=0)
    train_y_std = torch.std(train_y, dim=0)
    train_y = (train_y-train_y_mean)/train_y_std

    
    mll, model = initialize_model(train_x, train_y, train_y_var)

    fit_gpytorch_model(mll, retain_graph=True)

    sampler = SobolQMCNormalSampler(1024)
    qNEI = qNoisyExpectedImprovement(model, train_x, sampler)

    candidates = optimize_acqf(
        acq_function=qNEI,
        bounds=bounds,
        q=n_points,
        num_restarts=10,
        raw_samples=500
    )

    return candidates

In [292]:
from botorch.exceptions import InputDataWarning
import warnings
warnings.filterwarnings("ignore", category=InputDataWarning)

NUM_ITERATIONS = 50

train_x, train_y, train_y_var = generate_initial_data(n=10)
best_observed = torch.max(train_y)

for iteration in range(1, NUM_ITERATIONS + 1):    
        print("="*30)
        print("Iteration number = ", iteration)
        t0 = time.time()

        
        candidate = get_next_points(train_x, train_y, train_y_var, 0, bounds, 1)
        candidate_x = candidate[0]
        candidate_y, candidate_y_var = blackbox_func(candidate_x[0])

        train_x = torch.cat([train_x, candidate_x])
        train_y = torch.cat([train_y, torch.tensor(candidate_y).reshape(-1,1)])
        train_y_var = torch.cat([train_y_var, torch.tensor(candidate_y_var).reshape(-1,1)])
        best_observed = torch.max(train_y)

        i = torch.argmax(train_y)
        print(f"{train_x[i]=}")
        print(f"{train_y[i]=}")

        t = time.time()
        print(f"Got in {t-t0} seconds")

Iteration number =  1
train_x[i]=tensor([-0.3864,  1.9171])
train_y[i]=tensor([6.2480])
Got in 1.0935146808624268 seconds
Iteration number =  2
train_x[i]=tensor([-0.3864,  1.9171])
train_y[i]=tensor([6.2480])
Got in 0.966233491897583 seconds
Iteration number =  3
train_x[i]=tensor([-0.5077,  1.9991])
train_y[i]=tensor([7.8668])
Got in 1.4166040420532227 seconds
Iteration number =  4
train_x[i]=tensor([-0.5077,  1.9991])
train_y[i]=tensor([7.8668])
Got in 1.3167614936828613 seconds
Iteration number =  5
train_x[i]=tensor([-0.5077,  1.9991])
train_y[i]=tensor([7.8668])
Got in 1.757781982421875 seconds
Iteration number =  6
train_x[i]=tensor([-0.5077,  1.9991])
train_y[i]=tensor([7.8668])
Got in 1.3919730186462402 seconds
Iteration number =  7
train_x[i]=tensor([-0.5077,  1.9991])
train_y[i]=tensor([7.8668])
Got in 1.3517851829528809 seconds
Iteration number =  8
train_x[i]=tensor([-0.5077,  1.9991])
train_y[i]=tensor([7.8668])
Got in 1.249516248703003 seconds
Iteration number =  9
train

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\botorch\optim\fit.py:102: OptimizationWarning: `scipy_minimize` terminated with status 3, displaying original message from `scipy.optimize.minimize`: ABNORMAL_TERMINATION_IN_LNSRCH
  warn(


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.